IMPORTING LIBRARIES

In [ ]:
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import keras
import pathlib
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense
from IPython import display
import random



DEFINING DISCRIMINATOR MODEL

In [ ]:
def get_discriminator_model(upscale_factor=3, channels=1):
    

    input = (300,300,channels)
    model = Sequential()
    model.add(layers.Conv2D(64,kernel_size = 3, strides=2, padding="same",input_shape= input))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.2))
    model.add(layers.Conv2D(64,kernel_size = 3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.2))
    model.add(layers.Conv2D(32,kernel_size = 3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))

    optimz = keras.optimizers.Adam(learning_rate=0.01)
    model.compile(loss="binary_crossentropy",optimizer=optimz,metrics=['accuracy'])
    return model




                        DATA SOURCE FOR DISCRIMINATOR

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
root_dir = "/gdrive/MyDrive/dataset"
root_dir = pathlib.Path(root_dir)
print(root_dir)

/gdrive/MyDrive/dataset


                    SPLITTING DATA INTO VALIDATION AND TRAINING

In [ ]:
crop_size = (300, 300)
batch_size = 64
upscale_factor = 3
input_size = 300 

train_ds = image_dataset_from_directory(
    root_dir,
    batch_size=batch_size,
    image_size=crop_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
    label_mode=None,
)

valid_ds = image_dataset_from_directory(
    root_dir,
    batch_size=batch_size,
    image_size=crop_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    label_mode=None,
)


Found 5160 files belonging to 1 classes.
Using 4128 files for training.
Found 5160 files belonging to 1 classes.
Using 1032 files for validation.


                        SCALE IMAGES FROM 0-1

In [ ]:
def normalize(images):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images

# The map function applies the normalize function to each element in the train
# and test datasets
train_ds =  train_ds.map(normalize)

valid_ds  =  valid_ds.map(normalize)

# The first time you use the dataset, the images will be loaded from disk
# Caching will keep them in memory, making training faster
train_ds =  train_ds.cache()
valid_ds  =  valid_ds.cache()


def process_input(input, input_size, upscale_factor):
    input = tf.image.rgb_to_yuv(input)
    last_dimension_axis = len(input.shape) - 1
    y, u, v = tf.split(input, 3, axis=last_dimension_axis)
    return tf.image.resize(y, [input_size, input_size], method="area")


def process_target(input):
    input = tf.image.rgb_to_yuv(input)
    last_dimension_axis = len(input.shape) - 1
    y, u, v = tf.split(input, 3, axis=last_dimension_axis)
    return y


train_ds = train_ds.map(
    lambda x: (process_input(x, input_size=300, upscale_factor=3), process_target(x))
)
train_ds = train_ds.prefetch(buffer_size=32)

valid_ds = valid_ds.map(
    lambda x: (process_input(x, input_size=300, upscale_factor=3), process_target(x))
)
valid_ds = valid_ds.prefetch(buffer_size=32)
print(train_ds)#


TypeError: ignored

                                                REAL SAMPLES FOR DISCRIMINATOR

In [ ]:
def real_sample_generator(dataset, samples):
  rd = 0
  selected_real= 0 
  numbering = 0
  for x in dataset:
    y = np.asarray(x[1], dtype='object')
    rd = np.random.randint(0,int(y.shape[0]),int(samples))
    selected_real = y[rd].astype('float32')
    numbering = np.ones((samples,1)).astype('float32')
    print("...")
  return selected_real, numbering


        FAKE SAMPLES FOR DISCRIMINATOR

In [ ]:
def fake_sample_generator(sample_size):
    samples = np.random.rand(300*300*sample_size).astype('float32')
    samples = samples.reshape((sample_size,300,300,1))
    numbering = np.zeros((sample_size,1))
    return samples, numbering
    

            TRAINING THE DISCRIMINATOR

In [ ]:
def discriminator_training(model, dataset, iteration = 100, batch =256):
    half_batch = int(batch_size/2)
    for i in range(iteration):
        x_real,y_real = real_sample_generator(dataset, half_batch)
        print("finished")
        _,real_accuracy = model.train_on_batch(x_real,y_real)
        print("finished accuracy calculation")
        x_fake,y_fake = fake_sample_generator(half_batch)
        print("finished2")
        _,fake_accuracy = model.train_on_batch(x_fake,y_fake)
        print('>%d real= %.0f%% fake = %.0f%%'%(i+1,real_accuracy*100,fake_accuracy*100))


                                    INITIALIZING THE DISCRIMINATOR MODEL
                                    

In [ ]:
model = get_discriminator_model()
model.summary()
discriminator_training(model,train_ds)

Streaming output truncated to the last 5000 lines.
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
finished
(32, 300, 300, 1)
(32, 1)
finished accuracy calculation
>29 real= 88% fake = 100%
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
finished
(32, 300, 300, 1)
(32, 1)
finished accuracy calculation
>30 real= 100% fake = 0%
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
finished
(32, 300, 300, 1)
(32, 1)
finished accuracy calculation
>31 real= 100% fake = 100%
...
...
...
...
...
...
...
...
...
...
...
...
...
...
